# Obsługa wstrząsów w Prophet: Lockdowny COVID-19 jako jednorazowe święta

Ten notatnik demonstruje, jak modelować nieoczekiwane zdarzenia (tzw. "shocks") w Facebook Prophet, używając przykładu lockdownów COVID-19.

**Źródło:** [Prophet Documentation - Handling Shocks](https://facebook.github.io/prophet/docs/handling_shocks.html#treating-covid-19-lockdowns-as-a-one-off-holidays)

## Wprowadzenie

Lockdowny COVID-19 spowodowały nagłe i znaczące zmiany w wielu szeregach czasowych (np. ruch pieszy, dane sprzedażowe, dane mobilności). Prophet umożliwia modelowanie takich jednorazowych zdarzeń jako "świąt" (holidays), które występują tylko raz, ale mogą mieć wpływ przed i po dacie zdarzenia.

### Kluczowe koncepcje:
- **One-off holidays**: Święta, które występują tylko raz (w przeciwieństwie do regularnych świąt rocznych)
- **Windows**: Okresy przed i po święcie, które również są dotknięte wpływem zdarzenia
- **Lockdown effects**: Modelowanie wpływu restrykcji COVID-19 na dane

## 1. Import bibliotek

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Ustawienia wizualizacji
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Biblioteki zaimportowane!")

## 2. Generowanie przykładowych danych

Stworzymy symulowane dane reprezentujące dzienny ruch (np. liczba pieszych w centrum handlowym) z wyraźnym spadkiem podczas lockdownu.

In [ ]:
# Generowanie dat od stycznia 2019 do grudnia 2021
dates = pd.date_range(start='2019-01-01', end='2021-12-31', freq='D')

# Trend bazowy + sezonowość roczna
np.random.seed(42)
trend = np.linspace(100, 120, len(dates))
yearly_seasonality = 20 * np.sin(2 * np.pi * np.arange(len(dates)) / 365.25)
weekly_seasonality = 10 * np.sin(2 * np.pi * np.arange(len(dates)) / 7)
noise = np.random.normal(0, 5, len(dates))

# Wartości bazowe
y = trend + yearly_seasonality + weekly_seasonality + noise

# Symulacja wpływu lockdownu COVID-19
# Pierwszy lockdown: marzec-maj 2020
lockdown1_start = '2020-03-15'
lockdown1_end = '2020-05-31'
lockdown1_mask = (dates >= lockdown1_start) & (dates <= lockdown1_end)
y[lockdown1_mask] *= 0.3  # 70% spadek podczas lockdownu

# Drugi lockdown (łagodniejszy): listopad 2020 - styczeń 2021
lockdown2_start = '2020-11-01'
lockdown2_end = '2021-01-31'
lockdown2_mask = (dates >= lockdown2_start) & (dates <= lockdown2_end)
y[lockdown2_mask] *= 0.6  # 40% spadek podczas drugiego lockdownu

# Utworzenie DataFrame
df = pd.DataFrame({
    'ds': dates,
    'y': y
})

print(f"Wygenerowano {len(df)} dni danych")
print(f"Zakres dat: {df['ds'].min()} do {df['ds'].max()}")
df.head()

In [ ]:
# Wizualizacja danych
plt.figure(figsize=(15, 6))
plt.plot(df['ds'], df['y'], linewidth=1, alpha=0.7)
plt.axvspan(pd.to_datetime(lockdown1_start), pd.to_datetime(lockdown1_end), 
            alpha=0.2, color='red', label='Lockdown 1')
plt.axvspan(pd.to_datetime(lockdown2_start), pd.to_datetime(lockdown2_end), 
            alpha=0.2, color='orange', label='Lockdown 2')
plt.title('Symulowane dane ruchu z wpływem lockdownów COVID-19', fontsize=14, fontweight='bold')
plt.xlabel('Data')
plt.ylabel('Wartość')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Model bazowy (bez uwzględnienia lockdownów)

Najpierw stworzymy model Prophet bez uwzględnienia lockdownów, aby zobaczyć jak model radzi sobie bez tej informacji.

In [ ]:
# Model bazowy
print("=== MODEL BAZOWY (BEZ LOCKDOWNÓW) ===")
model_baseline = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)

model_baseline.fit(df)
print("Model bazowy wytrenowany!")

In [ ]:
# Prognoza dla modelu bazowego
future_baseline = model_baseline.make_future_dataframe(periods=180)  # 6 miesięcy do przodu
forecast_baseline = model_baseline.predict(future_baseline)

# Wizualizacja
fig = model_baseline.plot(forecast_baseline, figsize=(15, 6))
plt.title('Model bazowy - bez uwzględnienia lockdownów', fontsize=14, fontweight='bold')
plt.axvspan(pd.to_datetime(lockdown1_start), pd.to_datetime(lockdown1_end), 
            alpha=0.2, color='red', label='Lockdown 1')
plt.axvspan(pd.to_datetime(lockdown2_start), pd.to_datetime(lockdown2_end), 
            alpha=0.2, color='orange', label='Lockdown 2')
plt.legend()
plt.tight_layout()
plt.show()

## 4. Definiowanie lockdownów jako jednorazowych świąt

Kluczowa część: definiujemy lockdowny jako "święta", które występują tylko raz. Prophet pozwala na:
- Określenie dat święta
- Dodanie okien przed i po święcie (`lower_window` i `upper_window`)
- Grupowanie powiązanych zdarzeń

In [ ]:
# Definicja lockdownów jako świąt
lockdowns = pd.DataFrame([
    {
        'holiday': 'lockdown_1',
        'ds': '2020-03-15',
        'lower_window': 0,
        'upper_window': 77,  # liczba dni lockdownu (15 marca - 31 maja = 77 dni)
    },
    {
        'holiday': 'lockdown_2',
        'ds': '2020-11-01',
        'lower_window': 0,
        'upper_window': 91,  # liczba dni lockdownu (1 listopada - 31 stycznia = 91 dni)
    }
])

print("Zdefiniowane lockdowny:")
print(lockdowns)

### Wyjaśnienie parametrów:

- **`holiday`**: Nazwa święta (lockdownu)
- **`ds`**: Data rozpoczęcia święta
- **`lower_window`**: Liczba dni przed `ds`, które są dotknięte wpływem (0 = zaczyna się w `ds`)
- **`upper_window`**: Liczba dni po `ds`, które są dotknięte wpływem

Przykład: Jeśli lockdown trwa od 15 marca do 31 maja (77 dni), ustawiamy:
- `ds = '2020-03-15'`
- `lower_window = 0` (efekt zaczyna się 15 marca)
- `upper_window = 77` (efekt kończy się 77 dni po 15 marca)

## 5. Model z lockdownami

Teraz stworzymy model Prophet z uwzględnieniem lockdownów jako jednorazowych świąt.

In [ ]:
# Model z lockdownami
print("=== MODEL Z LOCKDOWNAMI ===")
model_with_lockdowns = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    holidays=lockdowns,  # Dodajemy lockdowny jako święta
    holidays_prior_scale=10.0  # Zwiększamy wpływ świąt (domyślnie 10.0)
)

model_with_lockdowns.fit(df)
print("Model z lockdownami wytrenowany!")

In [ ]:
# Prognoza dla modelu z lockdownami
future_with_lockdowns = model_with_lockdowns.make_future_dataframe(periods=180)
forecast_with_lockdowns = model_with_lockdowns.predict(future_with_lockdowns)

# Wizualizacja
fig = model_with_lockdowns.plot(forecast_with_lockdowns, figsize=(15, 6))
plt.title('Model z lockdownami - lepsze dopasowanie', fontsize=14, fontweight='bold')
plt.axvspan(pd.to_datetime(lockdown1_start), pd.to_datetime(lockdown1_end), 
            alpha=0.2, color='red', label='Lockdown 1')
plt.axvspan(pd.to_datetime(lockdown2_start), pd.to_datetime(lockdown2_end), 
            alpha=0.2, color='orange', label='Lockdown 2')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Komponenty modelu z lockdownami

Przyjrzyjmy się komponentom modelu, w tym wpływowi lockdownów.

In [ ]:
# Komponenty modelu
fig = model_with_lockdowns.plot_components(forecast_with_lockdowns, figsize=(15, 12))
plt.tight_layout()
plt.show()

### Analiza komponentów:

1. **Trend**: Ogólny trend wzrostowy/spadkowy w czasie
2. **Holidays**: Wpływ lockdownów (widoczne wyraźne spadki)
3. **Weekly**: Sezonowość tygodniowa
4. **Yearly**: Sezonowość roczna

In [ ]:
# Wyświetlenie szczegółów wpływu lockdownów
print("=== WPŁYW LOCKDOWNÓW NA MODEL ===")

# Filtrujemy prognozy dla okresów lockdownów
lockdown1_forecast = forecast_with_lockdowns[
    (forecast_with_lockdowns['ds'] >= lockdown1_start) & 
    (forecast_with_lockdowns['ds'] <= lockdown1_end)
]

lockdown2_forecast = forecast_with_lockdowns[
    (forecast_with_lockdowns['ds'] >= lockdown2_start) & 
    (forecast_with_lockdowns['ds'] <= lockdown2_end)
]

print(f"\nŚredni wpływ Lockdown 1: {lockdown1_forecast['holidays'].mean():.2f}")
print(f"Średni wpływ Lockdown 2: {lockdown2_forecast['holidays'].mean():.2f}")

print("\nUjemne wartości wskazują na spadek względem wartości bazowej.")

## 7. Porównanie modeli

Porównajmy prognozy obu modeli, aby zobaczyć różnicę.

In [ ]:
# Porównanie modeli
plt.figure(figsize=(16, 8))

# Dane rzeczywiste
plt.plot(df['ds'], df['y'], 'ko', markersize=2, alpha=0.3, label='Dane rzeczywiste')

# Prognoza modelu bazowego
plt.plot(forecast_baseline['ds'], forecast_baseline['yhat'], 
         linewidth=2, alpha=0.7, label='Model bazowy (bez lockdownów)', color='blue')

# Prognoza modelu z lockdownami
plt.plot(forecast_with_lockdowns['ds'], forecast_with_lockdowns['yhat'], 
         linewidth=2, alpha=0.7, label='Model z lockdownami', color='green')

# Zaznaczenie lockdownów
plt.axvspan(pd.to_datetime(lockdown1_start), pd.to_datetime(lockdown1_end), 
            alpha=0.2, color='red', label='Lockdown 1')
plt.axvspan(pd.to_datetime(lockdown2_start), pd.to_datetime(lockdown2_end), 
            alpha=0.2, color='orange', label='Lockdown 2')

plt.title('Porównanie: Model bazowy vs Model z lockdownami', fontsize=14, fontweight='bold')
plt.xlabel('Data')
plt.ylabel('Wartość')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Metryki porównawcze

Obliczmy metryki błędów dla obu modeli.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Wyciągnij prognozy dla historycznych dat
historical_baseline = forecast_baseline[forecast_baseline['ds'].isin(df['ds'])]
historical_with_lockdowns = forecast_with_lockdowns[forecast_with_lockdowns['ds'].isin(df['ds'])]

# Dopasuj kolejność
historical_baseline = historical_baseline.sort_values('ds').reset_index(drop=True)
historical_with_lockdowns = historical_with_lockdowns.sort_values('ds').reset_index(drop=True)
df_sorted = df.sort_values('ds').reset_index(drop=True)

# Oblicz metryki
print("=== PORÓWNANIE METRYK ===")
print("\nModel bazowy (bez lockdownów):")
mae_baseline = mean_absolute_error(df_sorted['y'], historical_baseline['yhat'])
rmse_baseline = np.sqrt(mean_squared_error(df_sorted['y'], historical_baseline['yhat']))
r2_baseline = r2_score(df_sorted['y'], historical_baseline['yhat'])
print(f"  MAE:  {mae_baseline:.2f}")
print(f"  RMSE: {rmse_baseline:.2f}")
print(f"  R²:   {r2_baseline:.4f}")

print("\nModel z lockdownami:")
mae_with_lockdowns = mean_absolute_error(df_sorted['y'], historical_with_lockdowns['yhat'])
rmse_with_lockdowns = np.sqrt(mean_squared_error(df_sorted['y'], historical_with_lockdowns['yhat']))
r2_with_lockdowns = r2_score(df_sorted['y'], historical_with_lockdowns['yhat'])
print(f"  MAE:  {mae_with_lockdowns:.2f}")
print(f"  RMSE: {rmse_with_lockdowns:.2f}")
print(f"  R²:   {r2_with_lockdowns:.4f}")

print("\n=== POPRAWA ===")
print(f"Redukcja MAE:  {((mae_baseline - mae_with_lockdowns) / mae_baseline * 100):.1f}%")
print(f"Redukcja RMSE: {((rmse_baseline - rmse_with_lockdowns) / rmse_baseline * 100):.1f}%")
print(f"Wzrost R²:     {((r2_with_lockdowns - r2_baseline) / r2_baseline * 100):.1f}%")

## 9. Zaawansowane zastosowania

### 9.1. Lockdowny z okresami przed i po

Możemy również modelować okresy przejściowe przed i po lockdownie.

In [ ]:
# Lockdowny z okresami przed i po
lockdowns_extended = pd.DataFrame([
    {
        'holiday': 'lockdown_1',
        'ds': '2020-03-15',
        'lower_window': -7,   # 7 dni przed lockdownem (przygotowania, panika)
        'upper_window': 84,   # 77 dni lockdownu + 7 dni po (powolne odbicie)
    },
    {
        'holiday': 'lockdown_2',
        'ds': '2020-11-01',
        'lower_window': -5,
        'upper_window': 96,   # 91 dni lockdownu + 5 dni po
    }
])

print("Lockdowny z okresami rozszerzonymi:")
print(lockdowns_extended)

In [ ]:
# Model z rozszerzonymi lockdownami
model_extended = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    holidays=lockdowns_extended,
    holidays_prior_scale=10.0
)

model_extended.fit(df)
future_extended = model_extended.make_future_dataframe(periods=180)
forecast_extended = model_extended.predict(future_extended)

# Wizualizacja
fig = model_extended.plot(forecast_extended, figsize=(15, 6))
plt.title('Model z rozszerzonymi lockdownami (okresy przed i po)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 9.2. Grupowanie lockdownów

Jeśli chcemy, aby wszystkie lockdowny miały podobny wpływ, możemy je pogrupować.

In [ ]:
# Grupowanie lockdownów - ta sama nazwa 'holiday'
lockdowns_grouped = pd.DataFrame([
    {
        'holiday': 'covid_lockdown',  # Ta sama nazwa
        'ds': '2020-03-15',
        'lower_window': 0,
        'upper_window': 77,
    },
    {
        'holiday': 'covid_lockdown',  # Ta sama nazwa
        'ds': '2020-11-01',
        'lower_window': 0,
        'upper_window': 91,
    }
])

print("Pogrupowane lockdowny (wspólny wpływ):")
print(lockdowns_grouped)

In [ ]:
# Model z pogrupowanymi lockdownami
model_grouped = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    holidays=lockdowns_grouped,
    holidays_prior_scale=10.0
)

model_grouped.fit(df)
future_grouped = model_grouped.make_future_dataframe(periods=180)
forecast_grouped = model_grouped.predict(future_grouped)

# Wizualizacja
fig = model_grouped.plot(forecast_grouped, figsize=(15, 6))
plt.title('Model z pogrupowanymi lockdownami (wspólny wpływ)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Podsumowanie i wnioski

### Kluczowe punkty:

1. **Jednorazowe święta (One-off holidays)**: Prophet pozwala na modelowanie zdarzeń, które występują tylko raz, takich jak lockdowny COVID-19.

2. **Okna wpływu**: Parametry `lower_window` i `upper_window` pozwalają na modelowanie okresów przed i po zdarzeniu.

3. **Poprawa dokładności**: Uwzględnienie lockdownów jako świąt znacząco poprawia dopasowanie modelu do danych historycznych.

4. **Grupowanie**: Możemy grupować podobne zdarzenia, aby dzieliły ten sam parametr wpływu.

5. **Prognozy**: Model z lockdownami lepiej radzi sobie z prognozowaniem przyszłości, ponieważ rozumie wpływ nieregularnych zdarzeń.

### Zastosowania praktyczne:

- **Analiza ruchu pieszego/samochodowego**: Modelowanie wpływu lockdownów na mobilność
- **Sprzedaż detaliczna**: Wpływ zamknięcia sklepów na sprzedaż
- **Turystyka**: Wpływ ograniczeń podróży na rezerwacje
- **Media społecznościowe**: Zmiany w aktywności użytkowników
- **Inne szoki**: Klęski żywiołowe, wydarzenia polityczne, itp.

### Parametry do dostrojenia:

- **`holidays_prior_scale`**: Kontroluje elastyczność modelu w dopasowywaniu się do świąt (większe = większy wpływ)
- **`lower_window` i `upper_window`**: Określają długość okresów wpływu
- **Grupowanie**: Decyzja czy każdy lockdown ma osobny wpływ, czy wspólny

## 11. Zastosowanie do rzeczywistych danych

Aby zastosować tę technikę do własnych danych:

```python
# 1. Zdefiniuj lockdowny/zdarzenia
my_events = pd.DataFrame([
    {
        'holiday': 'event_name',
        'ds': 'YYYY-MM-DD',  # Data rozpoczęcia
        'lower_window': 0,    # Dni przed
        'upper_window': X,    # Dni po
    }
])

# 2. Dodaj do modelu
model = Prophet(
    holidays=my_events,
    holidays_prior_scale=10.0  # Dostosuj w razie potrzeby
)

# 3. Trenuj i prognozuj
model.fit(df)
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)
```

## Zasoby dodatkowe

- [Dokumentacja Prophet - Handling Shocks](https://facebook.github.io/prophet/docs/handling_shocks.html)
- [Dokumentacja Prophet - Seasonality, Holiday Effects](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html)
- [Prophet GitHub](https://github.com/facebook/prophet)
- [Prophet Paper (Taylor & Letham, 2017)](https://peerj.com/preprints/3190/)